In [1]:
from src import Bayes, LSTM, BERT
import pandas as pd
import numpy as np
from torchinfo import summary
import calamancy
from torch import nn, optim, Tensor, LongTensor
from transformers import BertForSequenceClassification, BertTokenizer
from skorch.hf import HuggingfacePretrainedTokenizer


/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_lg' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn

## Loading Dataset

In [2]:
def read_csv_file(filename: str) -> pd.DataFrame:
    try:
        data = pd.read_csv(filename, lineterminator='\n', usecols=range(2))
        print("CSV file read successfully!")
        return data
    except FileNotFoundError:
        print("ERROR: File not found")
        exit(1)

dataset = read_csv_file('datasets/datasetall.csv')

dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


## Bernoulli Naive Bayes

In [3]:
Bayes.BayesPipeline

Pipeline(steps=[('tfidf', CountVectorizer()), ('bayes', BernoulliNB())])

In [4]:
X_transformed = Bayes.Vectorizer.fit_transform(dataset['text'])
X_transformed.shape

(28461, 45636)

In [5]:
print(X_transformed)

  (0, 4482)	1
  (0, 32400)	1
  (0, 2065)	1
  (0, 18173)	1
  (0, 8313)	1
  (0, 37041)	1
  (0, 23692)	1
  (0, 31562)	1
  (0, 43519)	1
  (1, 2065)	2
  (1, 37041)	3
  (1, 43519)	1
  (1, 13210)	1
  (1, 40923)	1
  (1, 44517)	1
  (1, 3331)	1
  (1, 20356)	2
  (1, 19996)	1
  (1, 36672)	1
  (1, 5675)	1
  (1, 21190)	2
  (1, 20984)	2
  (1, 19363)	1
  (1, 20834)	2
  (1, 31670)	1
  :	:
  (28459, 32033)	1
  (28459, 15162)	1
  (28459, 18055)	1
  (28459, 19801)	1
  (28459, 41882)	1
  (28459, 2239)	1
  (28459, 39117)	1
  (28459, 36621)	1
  (28460, 32705)	1
  (28460, 30713)	1
  (28460, 29015)	1
  (28460, 26347)	1
  (28460, 9124)	1
  (28460, 24958)	1
  (28460, 20573)	1
  (28460, 30422)	1
  (28460, 18975)	1
  (28460, 40806)	1
  (28460, 26219)	1
  (28460, 39129)	1
  (28460, 15777)	1
  (28460, 41031)	1
  (28460, 26430)	1
  (28460, 28503)	1
  (28460, 1847)	1


In [6]:
pd.DataFrame(
    zip(Bayes.Vectorizer.get_feature_names_out(), Bayes.Vectorizer.tf_),
    columns=['feature', 'IDF'],
)

AttributeError: 'CountVectorizer' object has no attribute 'tf_'

In [7]:
Bayes.BayesModel.fit(X_transformed, dataset['label'])

BernoulliNB()

## LSTM

In [8]:
LSTM.LstmPipeline

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.LSTM.LstmModel'>,
  module__hidden_size=400,
))])

In [9]:
summary(LSTM.LstmModel(), input_size=(1000, 200))

Layer (type:depth-idx)                   Output Shape              Param #
LstmModel                                [1000, 2]                 --
├─LSTM: 1-1                              [1000, 400]               2,246,400
├─Linear: 1-2                            [1000, 2]                 802
Total params: 2,247,202
Trainable params: 2,247,202
Non-trainable params: 0
Total mult-adds (G): 898.56
Input size (MB): 0.80
Forward/backward pass size (MB): 3.22
Params size (MB): 8.99
Estimated Total Size (MB): 13.00

### CalamanCy for tokenization and vectorization

In [10]:
Calamancy = calamancy.load("tl_calamancy_md-0.1.0")
Calamancy

/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)


In [17]:
tokenized_text = [
    text for text in
    Calamancy.pipe(dataset['text'][:20])
]

pd.DataFrame(tokenized_text)

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,Binay,:,Patuloy,ang,kahirapan,dahil,sa,maling,pamamahala,[,...,None,None,None,None,None,None,None,None,None,None
1,SA,GOBYERNONG,TAPAT,WELCOME,SA,BAGUO,ANG,LAHAT,!,Kulay,...,None,None,None,None,None,None,None,None,None,None
2,wait,so,ur,telling,me,Let,Leni,Lead,mo,pero,...,None,None,None,None,None,None,None,None,None,None
3,,[,USERNAME]wish,this,is,just,a,nightmare,that,could,...,None,None,None,None,None,None,None,None,None,None
4,doc,willie,ong,and,isko,sabunutan,po,None,None,None,...,None,None,None,None,None,None,None,None,None,None
5,Jeonghan,said,Let,Leni,Lead,!,!,Lenihae,ahahhdjfj,None,...,None,None,None,None,None,None,None,None,None,None
6,[,USERNAME,],[,USERNAME]and[USERNAME,],Hahahhahaha,SMNIpresidentialdebateLeni,Mas,Duwag,...,None,None,None,None,None,None,None,None,None,None
7,[,USERNAME,],Attendees,of,the,CNNPHVPDebate,are,Walden,Bello,...,None,None,None,None,None,None,None,None,None,None
8,Abscbn,News,Hoaxnews,ng,abias,cbend,baklas,poster,ng,mmda,...,None,None,None,None,None,None,None,None,None,None
9,NEWS,UPDATE,:,The,camp,of,VP,Leni,Robredo,says,...,is,unnecessary,.,'[USERNAME,],None,None,None,None,None


In [19]:
calamancy_result = [
    text.vector.reshape(1, -1)
    for text 
    in tokenized_text
]

# Concatenate all of them to form tensors of the right
# output shape.
calamancy_result = np.concatenate(calamancy_result)

pd.DataFrame(calamancy_result)

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.079959,0.030824,0.023937,0.077056,-0.016278,-0.213464,-0.033583,-0.129466,-0.295509,-0.336769,...,0.063488,0.263103,0.012206,-0.029511,0.423898,-0.299389,-0.295374,-0.069985,0.036437,-0.282058
1,-0.098400,-0.002249,0.276727,0.051350,-0.031479,0.004027,0.136961,-0.325066,0.048407,-0.353043,...,0.059415,0.348630,-0.128219,0.041348,0.014189,-0.230255,0.006952,-0.169405,0.153690,-0.381741
2,0.036431,-0.158402,0.274495,0.142066,0.137893,-0.024042,0.365713,-0.213699,0.030511,-0.351944,...,0.057017,0.135505,-0.285796,0.095381,0.037737,-0.077047,-0.194688,-0.001906,0.014809,-0.055193
3,-0.020916,-0.041349,0.302016,0.177881,0.241347,-0.235771,0.204203,-0.336757,-0.018283,-0.492876,...,0.119083,0.039149,-0.086413,-0.021435,-0.049193,-0.058399,-0.157710,0.141402,-0.201925,-0.075786
4,-0.119516,0.196659,-0.198787,-0.077372,0.622887,-0.081074,0.503828,-0.049475,0.076050,-0.174211,...,0.419373,0.268442,-0.197731,0.094939,0.392143,-0.017123,-0.237679,0.049520,-0.020215,-0.104863
5,-0.029591,-0.244930,0.097660,0.023090,-0.122789,-0.055445,-0.089991,-0.373714,0.079334,-0.225347,...,0.234361,0.270526,-0.221429,0.279023,0.466654,0.074242,-0.063258,0.185144,-0.041592,-0.270013
6,-0.290399,-0.052767,0.052822,0.073290,0.064098,-0.028226,-0.213334,-0.432583,0.006680,-0.681629,...,-0.039767,0.452986,0.135093,0.158477,0.552461,-0.567329,-0.951275,-0.063950,0.257712,-0.375186
7,0.014622,0.166466,0.225856,-0.080627,0.253168,-0.072141,0.085718,-0.387084,-0.023231,-0.415185,...,-0.031626,0.320854,0.041470,-0.107030,-0.059270,-0.073103,-0.036401,0.206450,-0.137248,-0.066766
8,-0.086514,0.060733,0.041636,-0.081284,0.082741,-0.125557,0.385665,-0.090126,-0.218261,-0.278023,...,0.174407,0.053065,-0.171381,-0.046194,0.228741,0.066286,-0.055473,0.083170,-0.100637,-0.069718
9,0.176639,-0.097223,0.090801,-0.032142,0.196946,-0.002278,0.253257,-0.208191,-0.029428,-0.372567,...,0.121507,0.138231,-0.321459,0.004063,-0.005993,-0.124676,-0.146378,0.181690,-0.175482,-0.205904


In [30]:
dataset['label'][:20]

0     0
1     0
2     1
3     0
4     0
5     0
6     1
7     0
8     1
9     0
10    0
11    0
12    1
13    1
14    0
15    1
16    1
17    0
18    1
19    1
Name: label, dtype: int64

### LSTM Layer

In [39]:
LSTM_INPUT_SIZE = 200
LSTM_HIDDEN_SIZE = 400
LSTM_OUTPUT_SIZE = 2
LSTM_NUM_LAYERS = 2

lstm_layer = nn.LSTM(
    LSTM_INPUT_SIZE,
    LSTM_HIDDEN_SIZE,
    LSTM_NUM_LAYERS,
    batch_first=True,
)

lstm_layer

LSTM(200, 400, num_layers=2, batch_first=True)

In [42]:
lstm_output, _ = lstm_layer(Tensor(calamancy_result))

In [44]:
pd.DataFrame(lstm_output.detach().numpy())

,0,1,2,3,4,5,6,7,8,9,...,390,391,392,393,394,395,396,397,398,399
0,-0.006848,0.015771,0.010692,-0.003858,0.010582,0.008932,0.000099,0.016196,-0.006349,-0.002703,...,-0.028811,-0.004469,-0.001727,0.015105,-0.003746,-0.016602,0.001922,0.017319,-0.005467,0.014621
1,-0.010850,0.025527,0.013016,-0.002878,0.013880,0.013564,0.000235,0.025177,-0.007978,0.001565,...,-0.046817,-0.004651,-0.001957,0.022598,-0.008415,-0.024361,-0.001754,0.029219,-0.005249,0.024511
2,-0.010654,0.028930,0.011545,0.003516,0.017411,0.015403,-0.002979,0.027395,-0.011294,0.009547,...,-0.053930,-0.005319,-0.000684,0.023960,-0.009157,-0.025524,-0.000380,0.037566,-0.006887,0.030381
3,-0.010484,0.035345,0.011292,0.012046,0.017548,0.015527,-0.008820,0.030164,-0.010472,0.012924,...,-0.059335,-0.005148,-0.000909,0.023990,-0.008414,-0.026476,-0.000486,0.041545,-0.008967,0.034236
4,-0.008598,0.037709,0.012929,0.016706,0.018869,0.016336,-0.012070,0.034518,-0.009251,0.017598,...,-0.062833,-0.006591,-0.000623,0.022965,-0.005757,-0.029461,0.000015,0.039833,-0.012315,0.036796
5,-0.013413,0.036513,0.016000,0.016436,0.014964,0.014808,-0.013999,0.033848,-0.008458,0.021361,...,-0.064448,-0.001836,-0.002050,0.021392,-0.003613,-0.032159,-0.003424,0.039998,-0.010311,0.042237
6,-0.017071,0.031282,0.014313,0.018560,0.008009,0.012881,-0.008967,0.034795,-0.004246,0.018917,...,-0.069302,-0.002360,0.000181,0.024292,-0.011299,-0.040109,-0.007195,0.046698,-0.002951,0.041583
7,-0.015015,0.034375,0.012964,0.018168,0.005430,0.010962,-0.009594,0.036771,-0.002959,0.017487,...,-0.071491,-0.001092,-0.001527,0.026783,-0.013183,-0.044036,-0.009448,0.048263,-0.000870,0.043555
8,-0.010909,0.037613,0.013823,0.018868,0.007676,0.009266,-0.011677,0.037975,-0.005863,0.020664,...,-0.072195,-0.002547,-0.004821,0.028436,-0.013900,-0.041257,-0.009221,0.046215,-0.002879,0.042427
9,-0.009212,0.041516,0.014701,0.020470,0.012359,0.010772,-0.013945,0.036786,-0.007852,0.021805,...,-0.070948,0.001672,-0.008100,0.027868,-0.013064,-0.040773,-0.009401,0.047903,-0.004370,0.040881


### Linear Layer

In [23]:
linear_layer = nn.Linear(LSTM_HIDDEN_SIZE, LSTM_OUTPUT_SIZE)

linear_layer

Linear(in_features=400, out_features=2, bias=True)

In [46]:
lstm_linear_output = linear_layer(lstm_output)

pd.DataFrame(lstm_linear_output.detach().numpy())

,0,1
0,0.026161,-0.017506
1,0.020555,-0.021444
2,0.020684,-0.022414
3,0.019620,-0.022225
4,0.016548,-0.022006
5,0.015566,-0.023021
6,0.014065,-0.025210
7,0.011853,-0.025951
8,0.012732,-0.026139
9,0.014234,-0.024414


### Loss Function

In [47]:
loss_function = nn.CrossEntropyLoss()

loss_function

CrossEntropyLoss()

In [56]:
lstm_loss = loss_function(lstm_linear_output, LongTensor(dataset['label'][:20]))

lstm_loss

tensor(0.6917, grad_fn=<NllLossBackward0>)

In [ ]:
lstm_loss.backward()

### Optimizer

In [58]:
lstm_optimizer = optim.Adam(lstm_layer.parameters(), lr=0.015, weight_decay=0.00001)

lstm_optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.015
    maximize: False
    weight_decay: 1e-05
)

In [59]:
lstm_optimizer.step()

## mBERT

In [62]:
BERT.BertPipeline

Pipeline(steps=[('tokenizer',
                 HuggingfacePretrainedTokenizer(max_length=255,
                                                tokenizer='bert-base-multilingual-cased')),
                ('bert',
                 <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.BERT.BertModel'>,
))])

### BERT Tokenizer

In [116]:
bert_tokenizer = BertTokenizer.from_pretrained(
    BERT._model_name,
)

bert_tokenizer

BertTokenizer(name_or_path='bert-base-multilingual-cased', vocab_size=119547, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [97]:
bert_tokenized = bert_tokenizer(
    list(dataset['text'][:20]),
    pad_to_max_length=True,
    max_length=255,
)

/home/nate/miniconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [98]:
pd.DataFrame(bert_tokenized['input_ids'])

,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
0,101,50754,13998,131,24714,22540,10157,10488,10730,21840,...,0,0,0,0,0,0,0,0,0,0
1,101,21318,41525,11274,14703,24093,67292,34065,91075,52099,...,0,0,0,0,0,0,0,0,0,0
2,101,83279,10380,10399,61758,10911,17144,97413,10116,48860,...,0,0,0,0,0,0,0,0,0,0
3,101,164,10808,24093,36742,42424,166,66069,10531,10124,...,0,0,0,0,0,0,0,0,0,0
4,101,67779,11337,10400,10135,10240,10111,10124,10440,10148,...,0,0,0,0,0,0,0,0,0,0
5,101,13796,13891,11781,12415,17144,97413,10116,48860,106,...,0,0,0,0,0,0,0,0,0,0
6,101,164,10808,24093,36742,42424,166,164,10808,24093,...,0,0,0,0,0,0,0,0,0,0
7,101,164,10808,24093,36742,42424,166,11699,58503,10171,...,0,0,0,0,0,0,0,0,0,0
8,101,56386,10350,71136,12219,15475,10686,72078,10743,11357,...,0,0,0,0,0,0,0,0,0,0
9,101,109023,53197,36382,29833,131,10117,16700,10108,100188,...,0,0,0,0,0,0,0,0,0,0


In [99]:
pd.DataFrame(bert_tokenized['attention_mask'])

,0,1,2,3,4,5,6,7,8,9,...,245,246,247,248,249,250,251,252,253,254
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
5,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
8,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


### mBERT Transformer

In [100]:
bert_model = BERT._model

bert_model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [105]:
bert_output = bert_model(
    LongTensor(bert_tokenized['input_ids']), 
    Tensor(bert_tokenized['attention_mask']),
)

bert_output

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.1311, -0.0341],
        [ 0.1012, -0.0527],
        [ 0.1273,  0.0391],
        [ 0.1370, -0.0109],
        [ 0.1438,  0.0340],
        [ 0.0969,  0.0443],
        [ 0.1153,  0.0068],
        [ 0.1757, -0.0918],
        [ 0.1408, -0.0344],
        [ 0.2511, -0.0544],
        [ 0.2031, -0.0792],
        [ 0.1136, -0.0541],
        [ 0.1312, -0.0255],
        [ 0.0987, -0.0781],
        [ 0.0622, -0.0133],
        [ 0.1800,  0.0519],
        [ 0.1252, -0.0098],
        [ 0.1253, -0.0208],
        [ 0.1585,  0.0436],
        [ 0.1204,  0.0718]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [107]:
pd.DataFrame(bert_output.logits.detach().numpy())

,0,1
0,0.131123,-0.034055
1,0.101174,-0.052707
2,0.127322,0.039107
3,0.136966,-0.010851
4,0.143763,0.034024
5,0.096941,0.044273
6,0.115334,0.006848
7,0.175718,-0.091790
8,0.140783,-0.034433
9,0.251128,-0.054372


### Loss Function

In [108]:
loss_function_bert = nn.CrossEntropyLoss()

loss_function_bert

CrossEntropyLoss()

In [110]:
bert_loss = loss_function_bert(bert_output.logits, LongTensor(dataset['label'][:20]))

bert_loss

tensor(0.6780, grad_fn=<NllLossBackward0>)

In [111]:
bert_loss.backward()

### Optimizer

In [112]:
bert_optimizer = optim.Adam(bert_model.parameters(), lr=0.00001, weight_decay=0.01)

bert_optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 1e-05
    maximize: False
    weight_decay: 0.01
)

In [113]:
bert_optimizer.step()